In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Import other libraries as you need them here.
#### You will need to re-run all or just this one cell if you add more imports later. You may also ignore this and import as you need later.

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
 
from sklearn.model_selection import train_test_split
 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

#### (1)
Load your DF. Display the first 5 rows.

In [ ]:
df = pd.read_csv('/kaggle/input/diamonds/diamonds.csv')
print(df.head())

#### (2)
Display the column names only.

In [ ]:
print(df.columns)

#### (3)

Data clean up time.

First, let's re-index all of the columns so that price becomes the final column. While doing so, we will remove the excess column by not placing in the list to replace.

Display the df.

In [ ]:
df = df.reindex( columns=['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z', 'price'] )  
print(df)

#### (4)

Let's take a look at the current data graphically. This will be useful later when we want to clean up some potential misleading data.  

Seaborn is a library designed for some amazing data visualizations.  
Import: seaborn as sns  
Then use: sns.pairplot( your data frame, hue='cut', plot_kws={'s':1} )  
This will take a bit of time to run. You can try a different column for the hue, color or clarity should work well. The plot_kws is to make the size of the individual plots be smaller.  
After running it and studying it you may want to open the image in a new tab then comment this line out while you work on the rest of the project so you do not run it again. Uncomment it before submitting.


In [ ]:
sns.pairplot(df, hue='cut', plot_kws={'s':1})


#### (5)

Some early project questions:

1) Which two potential features (variables) may not be useful for predicting price?

2) Which potential features look like they may be most useful for predicing price?

3) Can you identify some potential incorrect data that needs to be removed? Name a few features and some ranges we may want to remove. (example: remove data with age > 100 )


In [ ]:
#1. Im not really sure the purpose of x, y, or z but also the depth seems extraneous and the cut variable.
#2. The price and carat seem most useful for predicting price.
#3. When the price is lower, all atributes seem to be more random and dispersed. When the cut is ideal or premium.

#### (6)

Display all of the description statistics of the df.

In [ ]:
print(df.describe())

#### (7)

You should notice that some of the columns are missing in the statistics and were also missing in the PairGrid. This is because these are categorical data instead of numerical. Most machine learning algorithms will not be able to work with categorical data without converting the data first into numerical values. There are libraries which can convert categories to numbers automatically for you, but they will just assign a number to each category with no regards to the 'value' of one category over another if the categories could be sorted. For instance, Bristow may get a 0, Gainesville may get a 1, Haymarket may get a 2. The order here could be arbitrary or alphabetic and it would not matter.  

Some categorical values could be ordered. In diamonds, there is an order to the cut, color, and clarity values. One is 'better' than another.  

Convert your categorical data into numerical data.  
Fair < Good < Very Good < Premium < Ideal  
J < I < H < ... < D  
I1 < SI2 < SI1 < VS2 < VS1 < VVS2 < VVS1 < IF < FL

Display your DF to check the categories converted.

In [ ]:
df['cut'].loc[df['cut'] == 'Fair'] = 1
df['cut'].loc[df['cut'] == 'Good'] = 2
df['cut'].loc[df['cut'] == 'Very Good'] = 3
df['cut'].loc[df['cut'] == 'Premium'] = 4
df['cut'].loc[df['cut'] == 'Ideal'] = 5
df['color'].loc[df['color'] == 'E'] = 1
df['color'].loc[df['color'] == 'I'] = 2
df['color'].loc[df['color'] == 'J'] = 3
df['color'].loc[df['color'] == 'H'] = 4
df['color'].loc[df['color'] == 'F'] = 5
df['color'].loc[df['color'] == 'G'] = 6
df['color'].loc[df['color'] == 'D'] = 7
df['clarity'].loc[df['clarity'] == 'I1'] = 1
df['clarity'].loc[df['clarity'] == 'SI2'] = 2
df['clarity'].loc[df['clarity'] == 'SI1'] = 3
df['clarity'].loc[df['clarity'] == 'VS2'] = 4
df['clarity'].loc[df['clarity'] == 'VS1'] = 5
df['clarity'].loc[df['clarity'] == 'VVS2'] = 6
df['clarity'].loc[df['clarity'] == 'VVS1'] = 7
df['clarity'].loc[df['clarity'] == 'IF'] = 8
df['clarity'].loc[df['clarity'] == 'FL'] = 9
df['cut'] = df['cut'].astype('int64')
df['color'] = df['color'].astype('int64')
df['clarity'] = df['clarity'].astype('int64')
print(df.dtypes)

#### (8)

Let's deal with some of the outliers you may have noticed now. These may be explainable, but we are going to assume they are data that will interfere with our predictions. You should notice a few points in the y and z features that are definitely not consistent. There are also diamonds showing x/y/z values as 0, which present incomplete data we cannot use. Figure out a way to remove a good portion of these pieces of data that will interfere with our predictions.

Re-display all the main statistics of your DF.



In [ ]:
df.loc[ (df.x == 0) | (df.y == 0) | (df.z == 0) ] = None
df = df.dropna()
print(df.reset_index(inplace=True))

#### (9)
Display the correlations of your DF, sorted in increasing order. You can either display all of them or limit the display only to the price column, which is the only correlation that will ultimately matter for our predictions.

In [ ]:
print(df.corr(method = 'pearson').sort_values('price'))

#### (10)
Diamonds are often talked about with their "Four C's". Cut, Color, Clarity, and Carat. According to these correlations, rank the "Four C's" from most important to least important in price. (Think for a moment on this one!)  
  
Most Important  
1)  
2)  
3)  
4)  
Least Important  

In [ ]:
#1. Clarity
#2. Carat (i had to look up what this one is, but it's the weight if i'm not mistaken)
#3. Color
#4. Cut

#### (11)
Now we will look a bit more at those correlations as a whole.  

A heatmap can be used to display the different correlations in a graphical way. A correlation can be anywhere from -1 to 1. A correlation of 0 means there is almost nothing connecting one variable to another. Closer to 1 means that it is almost guaranteed that as one variable goes up, another goes up almost linearly the same. Closer to -1 means as one goes up, another goes down almost linearly.

To display a heatmap, use: sns.heatmap( your dataframe's correlation ) or sns.heatmap( your dataframe's correlations, annot=True). Try out both to see the difference.

In [ ]:
sns.heatmap(df.corr(method = 'pearson'), annot = True)

#### (12)
These are similar questions to the earlier ones.

1) Which features seem to be the best to be used to predict the price of a diamond?  

Carat

2) Which features may need to be thrown out during predictions?  

Color, Depth,or Clarity

3) There are several variables that are correlated so highly that perhaps it would be okay to remove some of them to cut down on computing times. Which variable(s) may you be able to cut and which variable(s) would you keep because it covers the same overall trends?

The X, Y, and Z columns all havea  similar coorelation to one another. Clarity is an attribute we ought to keep. 


#### (13)
Build a linear regression model.
Choose all of the features (aside from price) for your X values.
Choose price for your y values.

Split your data into two equal halves for training and validating.
Run your model on your training set.
Predict your y values based on your validation X values.

What is your mean absolute error?  


In [ ]:
linReg = LinearRegression()
X = df[['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']]
y = df.price
train_X, test_X, train_y, test_y = train_test_split(X, y)
linReg.fit(train_X, train_y)
linReg_pred = linReg.predict( test_X )

print(mean_absolute_error(test_y, linReg_pred))

#### (14)
Display the coefficients your linear model computed in a nice chart showing the variable name and the computed coefficient.



In [ ]:
linReg_coefs = linReg.coef_
coef_df = pd.DataFrame( columns = ['Feature', 'Coefficient'] )
coef_df['Feature'] = ['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']
coef_df['Coefficient'] = linReg_coefs
print(coef_df)

#### (15)

Hard-code a line of code using each of your found coefficients to calculate/display the predicted price of the following diamond:

Carat: 0.8  
Cut: Very Good  
Color: F  
Clarity: VVS2   
Depth: 61.8  
Table: 57.5  
x: 5.7  
y: 5.7  
z: 3.5  

In [ ]:
coef_df['diamond_1'] = [0.8, 3, 5, 6, 61.8, 57.5, 5.7, 5.7, 3.5]
print( sum( coef_df['Coefficient'] * [0.8, 3, 5, 6, 61.8, 57.5, 5.7, 5.7, 3.5] ) )

#### (16)
Run another LinearRegression model, but this time use only the "Four C's" as your features. Display your mean absolute error.

In [ ]:
linReg16 = LinearRegression()
X = df[['carat', 'cut', 'color', 'clarity']]
y = df.price
train_X, test_X, train_y, test_y = train_test_split(X, y)
linReg16.fit(train_X, train_y)
linReg_pred16 = linReg16.predict( test_X )

print(mean_absolute_error(test_y, linReg_pred16))

#### (17)
Try again, this time only with carat.

In [ ]:
linReg17 = LinearRegression()
X = df[['carat']]
y = df.price
train_X, test_X, train_y, test_y = train_test_split(X, y)
linReg17.fit(train_X, train_y)
linReg_pred17 = linReg17.predict( test_X )

print(mean_absolute_error(test_y, linReg_pred17))

#### (18)
One more time with carat, x, y, and z (the four highest correlating features compared to price).

In [ ]:
linReg18 = LinearRegression()
X = df[['carat', 'x', 'y', 'z']]
y = df.price
train_X, test_X, train_y, test_y = train_test_split(X, y)
linReg18.fit(train_X, train_y)
linReg_pred18 = linReg18.predict( test_X )

print(mean_absolute_error(test_y, linReg_pred18))

#### (19)
Compare the last three models used which utilized less features than the first.

1) Is there anything that seems non-intuitive about which of these three sets is best?

I dont understand why the numbers change every I rerun the code. 

2) Can you attempt to explain why it is the best of these three?

I feel the set using carat, x, y, and z yielded the most accurate prediction because they are most closely coorelated. Additionally, there is just the right amount of data for a smooth curve without the extra noise that utilizing all the columns would add.

3) Using all possible features gave the best result (hopefully). Assuming you had no concern over how long your model took to execute, can you think of a reason why you may want to leave a feature out of your model?

Certain columns are inversely related to the price of a diamond and would make the answer inaccurate.

#### (20)
Create a copy of your dataframe, then add a new column 'predictions' and fill it with the results from predicting your model on the entire original X with all of the columns. (If you re-used the same variable for the last few regression models again, make sure you reset the model back to using all of the columns).  

Create another new column called abs_error. Set this column to equal the absolute value of the price column minus the predictions column to show how far off the prediction was.

Display your DF.

In [ ]:
df2 = df.copy()
df2 = df2[['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']]
df2['predictions'] = linReg.predict(df2)
df2['price'] = df['price'].copy()
df2['abs_error'] = abs(df2['price']) - df2['predictions']
print(df2)

#### (21)
Display the full statistics on your DF again. Look particularly at the mean on the abs_error column. Does it match the number calculated above or is it slightly off? If it is off, can you explain why that may be?

In [ ]:
#Certain carats yield wild results, even negative numbers so I feel the Linear Regression may be calculated wrong

#### (22)
Display a scatter plot comparing your price column to your predictions column. Ideally this would be an almost perfect line. You may want to make the size of the individual plotted points to be smaller. Title your plot with the type of regression used (Linear Regression in this case).

In [ ]:
df2.plot.scatter(x = 'price', y = 'predictions', s=0.01, title = 'Linear_Regression')

#### (23)
Now run the model as a decision tree using all variables. Again show the mean absolute error.

In [ ]:
decTree = DecisionTreeRegressor(criterion = 'mse', max_depth = 5)
X = df[['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']]
y = df.price
train_X, test_X, train_y, test_y = train_test_split(X, y)
decTree.fit(train_X, train_y)
decTree_pred = decTree.predict( test_X )

print(mean_absolute_error(test_y, decTree_pred))

#### (24)
Display a scatter plot comparing your price column to your predictions column. Ideally this would be an almost perfect line. You may want to make the size of the individual plotted points to be smaller. Title your plot with the type of regression used.

In [ ]:
df2 = df.copy()
df2 = df2[['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']]
df2['predictions'] = decTree.predict(df2)
df2['price'] = df['price'].copy()
df2['abs_error'] = abs(df2['price']) - df2['predictions']
print(df2)
df2.plot.scatter(x = 'price', y = 'predictions', s = 0.01, title = 'Deciduos Tree')

#### (25)
Now run the model as a random forest.

In [ ]:
randForrest = RandomForestRegressor(criterion = 'mse', max_depth = 5)
X = df[['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']]
y = df.price
train_X, test_X, train_y, test_y = train_test_split(X, y)
randForrest.fit(train_X, train_y)
randForrest_pred = randForrest.predict( test_X )

print(mean_absolute_error(test_y, randForrest_pred))

#### (26)
Display a scatter plot comparing your price column to your predictions column. Ideally this would be an almost perfect line. You may want to make the size of the individual plotted points to be smaller. Title your plot with the type of regression used.


In [ ]:
df2 = df.copy()
df2 = df2[['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']]
df2['predictions'] = randForrest.predict(df2)
df2['price'] = df['price'].copy()
df2['abs_error'] = abs(df2['price']) - df2['predictions']
print(df2)
df2.plot.scatter(x = 'price', y = 'predictions', s = 0.01, title = 'Random Forrest')

#### (27)
Choose the model with the least mean absolute error. Update the prediction column and the abs_error column to use this model's predictions.
Display your final df.

In [ ]:
linReg = LinearRegression()
X = df[['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']]
y = df.price
train_X, test_X, train_y, test_y = train_test_split(X, y)
linReg.fit(train_X, train_y)
linReg_pred = linReg.predict( test_X )
df2 = df.copy()
df2 = df2[['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z']]
df2['predictions'] = linReg.predict(df2)
df2['price'] = df['price'].copy()
df2['abs_error'] = abs(df2['price']) - df2['predictions']
df2.plot.scatter(x = 'price', y = 'predictions', s=0.01, title = 'Linear_Regression')